In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [124]:
import findspark 
findspark.init()
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
from pyspark.sql.types import *

spark = SparkSession.builder.appName("Spark ML example on titanic data").getOrCreate()


In [125]:
path_and_file = './data_japan_house/13_Tokyo_20161_20182.csv'
sdf_tokyo = spark.read.csv(path_and_file,header='True',inferSchema='True',encoding='SHIFT-JIS')
sdf_tokyo.show(1)

+---+---------+---+-------+-----+-----+---+------+------+---------+----+----+---+------+-----+---+----+-----+-----+---------+-------+-------+-------+-------+----+----+---+----------+----+------+
| No|       種類| 地域|市区町村コード|都道府県名|市区町村名|地区名|最寄駅：名称|最寄駅：距離|    取引価格9| 坪単価| 間取り| 面積|取引価格13|土地の形状| 間口|延床面積|  建築年|建物の構造|       用途|今後の利用目的|前面道路：方位|前面道路：種類|前面道路：幅員|都市計画|建ぺい率|容積率|      取引時点|  改装|取引の事情等|
+---+---------+---+-------+-----+-----+---+------+------+---------+----+----+---+------+-----+---+----+-----+-----+---------+-------+-------+-------+-------+----+----+---+----------+----+------+
|  1|宅地(土地と建物)|商業地|  13101|  東京都| 千代田区|飯田橋|   飯田橋|     3|250000000|null|null| 80|  null| ほぼ台形|6.8| 330|昭和61年|   ＲＣ|住宅、事務所、店舗|    事務所|     南西|     区道|    8.0|商業地域|  80|500|平成30年第２四半期|null|  null|
+---+---------+---+-------+-----+-----+---+------+------+---------+----+----+---+------+-----+---+----+-----+-----+---------+-------+-------+-------+-------+----+----+---+----------+----+------+
only showing top 1 row



In [126]:
path_and_file = './data_japan_house/14_Kanagawa Prefecture_20161_20182.csv'
sdf_kanagawa = spark.read.csv(path_and_file,header='True',inferSchema='True',encoding='SHIFT-JIS')
sdf_kanagawa = sdf_kanagawa.filter(col('市区町村名').contains('横浜')|col('市区町村名').contains('川崎'))
sdf_kanagawa.show()

+---+---------+----+-------+-----+------+---+------+------+--------+------+------+---+------+-----+----+----+-----+-----+----+-------+-------+-------+-------+-----+----+---+----------+----+------+
| No|       種類|  地域|市区町村コード|都道府県名| 市区町村名|地区名|最寄駅：名称|最寄駅：距離|   取引価格9|   坪単価|   間取り| 面積|取引価格13|土地の形状|  間口|延床面積|  建築年|建物の構造|  用途|今後の利用目的|前面道路：方位|前面道路：種類|前面道路：幅員| 都市計画|建ぺい率|容積率|      取引時点|  改装|取引の事情等|
+---+---------+----+-------+-----+------+---+------+------+--------+------+------+---+------+-----+----+----+-----+-----+----+-------+-------+-------+-------+-----+----+---+----------+----+------+
|  1|宅地(土地と建物)| 住宅地|  14101| 神奈川県|横浜市鶴見区|朝日町|    浅野|    11|36000000|  null|  null| 60|  null|  長方形| 5.5|  95|平成29年|   木造|  住宅|     住宅|      南|     私道|    4.2|準工業地域|  60|200|平成29年第４四半期|null|  null|
|  2|宅地(土地と建物)| 住宅地|  14101| 神奈川県|横浜市鶴見区|朝日町|    浅野|    11|31000000|  null|  null| 50|  null|  長方形| 3.6|  85|平成29年|   木造|  住宅|     住宅|      東|     市道|    4.2|準工業地域|  60|200|平成29年第３四半期|null|  null|
|  3| 中古マンション等|

In [127]:
sdf = sdf_tokyo.unionAll(sdf_kanagawa)

In [128]:
describe_result = sdf.describe().toPandas()
describe_result

,summary,No,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離,...,今後の利用目的,前面道路：方位,前面道路：種類,前面道路：幅員,都市計画,建ぺい率,容積率,取引時点,改装,取引の事情等
0,count,103922,103922,55002,103922,103922,103922,103922,103554,103435,...,94732,55000,54450,54156,103261,102762,102762,103922,43075,6503
1,mean,31192.466138065087,None,None,13409.291410865842,None,None,None,None,9.82792388629528,...,None,None,None,6.157563335549175,None,61.808450594577764,251.76650902084427,None,None,None
2,stddev,21647.94637294824,None,None,437.6623099845058,None,None,None,None,6.1965671774261635,...,None,None,None,4.548122470310611,None,12.944754147308164,157.21948728536037,None,None,None
3,min,1,中古マンション等,住宅地,13101,東京都,あきる野市,あかね台,あざみ野,0,...,その他,北,区画街路,1.0,商業地域,30,50,平成28年第１四半期,改装済,その他事情有り
4,max,75233,農地,工業地,14137,神奈川県,青梅市,（大字なし）,黒川(神奈川),9,...,店舗,西,都道,90.0,都市計画区域外,80,1300,平成30年第２四半期,未改装,隣地の購入、関係者間取引


In [129]:
sdf.printSchema()

root
 |-- No: integer (nullable = true)
 |-- 種類: string (nullable = true)
 |-- 地域: string (nullable = true)
 |-- 市区町村コード: integer (nullable = true)
 |-- 都道府県名: string (nullable = true)
 |-- 市区町村名: string (nullable = true)
 |-- 地区名: string (nullable = true)
 |-- 最寄駅：名称: string (nullable = true)
 |-- 最寄駅：距離: string (nullable = true)
 |-- 取引価格9: long (nullable = true)
 |-- 坪単価: integer (nullable = true)
 |-- 間取り: string (nullable = true)
 |-- 面積: string (nullable = true)
 |-- 取引価格13: integer (nullable = true)
 |-- 土地の形状: string (nullable = true)
 |-- 間口: string (nullable = true)
 |-- 延床面積: string (nullable = true)
 |-- 建築年: string (nullable = true)
 |-- 建物の構造: string (nullable = true)
 |-- 用途: string (nullable = true)
 |-- 今後の利用目的: string (nullable = true)
 |-- 前面道路：方位: string (nullable = true)
 |-- 前面道路：種類: string (nullable = true)
 |-- 前面道路：幅員: double (nullable = true)
 |-- 都市計画: string (nullable = true)
 |-- 建ぺい率: integer (nullable = true)
 |-- 容積率: integer (nullable = true)
 |-- 取引時点:

全部関係あるが、重要じゃなさそうな奴を削除

In [130]:
sdf = sdf.drop( 'No','都道府県名','地域','市区町村コード','地区名',
                '坪単価',
               '土地の形状',
               '間口',
               '用途',
               '今後の利用目的',
                '前面道路：方位',
               '最寄駅：名称',
                '前面道路：種類',
                '前面道路：幅員',
                '取引時点',
                '取引価格13',
                '改装',
                '取引の事情等')

欠損値処理  

欠損値確認

In [131]:
sdf.count()

103922

In [132]:
sdf.describe().toPandas()

,summary,種類,市区町村名,最寄駅：距離,取引価格9,間取り,面積,延床面積,建築年,建物の構造,都市計画,建ぺい率,容積率
0,count,103922,103922,103435,103922,46848,103922,36294,83196,83326,103261,102762,102762
1,mean,None,None,9.82792388629528,6.041636920767499E7,None,111.29266503903423,150.7402683819452,None,None,None,61.808450594577764,251.76650902084427
2,stddev,None,None,6.1965671774261635,2.4223598810029456E8,None,153.21097015129263,187.96804853242497,None,None,None,12.944754147308164,157.21948728536037
3,min,中古マンション等,あきる野市,0,1000,オープンフロア,10,10,平成10年,ブロック造,商業地域,30,50
4,max,農地,青梅市,9,30000000000,６ＬＤＫ＋Ｓ,990,990,昭和63年,ＳＲＣ、ＲＣ,都市計画区域外,80,1300


ないものを未知に  

In [133]:
sdf = sdf.filter(
    col('最寄駅：名称').isNotNull()&
    col('最寄駅：距離').isNotNull()&
    col('都市計画').isNotNull()&
    col('建ぺい率').isNotNull()&
    col('容積率').isNotNull()&
    col('建築年').isNotNull()
    
)
sdf.count()

82457

In [134]:
for c in sdf.columns:
    print(c,'の欠損値の数',sdf.filter(col(c).isNull()).count())

種類 の欠損値の数 0
市区町村名 の欠損値の数 0
最寄駅：距離 の欠損値の数 0
取引価格9 の欠損値の数 0
間取り の欠損値の数 37353
面積 の欠損値の数 0
延床面積 の欠損値の数 47254
建築年 の欠損値の数 0
建物の構造 の欠損値の数 1446
都市計画 の欠損値の数 0
建ぺい率 の欠損値の数 0
容積率 の欠損値の数 0


In [135]:
features_to_fill =[
    '間取り',
    '延床面積',
    '建築年',
    '建物の構造']

sdf = sdf.fillna('未知',subset=features_to_fill)

In [136]:
sdf.groupby('最寄駅：距離').count().show()

+-------+-----+
| 最寄駅：距離|count|
+-------+-----+
|      7| 4776|
|     15| 2703|
|     11| 4351|
|1H30?2H|   27|
|     29|  468|
|      3| 5188|
|      8| 6170|
|     28|  577|
|     22|   89|
|     16| 2292|
|      0|  329|
|      5| 5820|
|     18| 1846|
|     27|   21|
|     17|  260|
|     26|  604|
|      6| 6558|
|     19| 1581|
|     23|  951|
|     25|  829|
+-------+-----+
only showing top 20 rows



In [137]:
li = ['1H30?2H','1H?1H30','30分?60分','2H?']
sdf = sdf.filter(~(col('最寄駅：距離').isin(li)))

In [138]:
sdf.filter(col('最寄駅：距離').isNull()).count()

0

駅までの距離を解決

間取りにはnullが入っている  
面積で補間したい

In [139]:
sdf.select('面積').describe().show()

+-------+------------------+
|summary|                面積|
+-------+------------------+
|  count|             78947|
|   mean| 88.49092730431254|
| stddev|105.63867365237994|
|    min|                10|
|    max|               990|
+-------+------------------+



In [140]:
sdf = sdf.filter(~(col('面積').contains('以上')))
sdf = sdf.filter(~(col('延床面積').contains('以上')))

500以上は広すぎ、

In [141]:
sdf = sdf.filter(~(col('面積')>500))

残りも、わからないのが未知

In [142]:
sdf.show()

+---------+-----+------+---------+----+---+----+-----+-----+----+----+---+
|       種類|市区町村名|最寄駅：距離|    取引価格9| 間取り| 面積|延床面積|  建築年|建物の構造|都市計画|建ぺい率|容積率|
+---------+-----+------+---------+----+---+----+-----+-----+----+----+---+
|宅地(土地と建物)| 千代田区|     3|250000000|  未知| 80| 330|昭和61年|   ＲＣ|商業地域|  80|500|
| 中古マンション等| 千代田区|     3| 22000000|  １Ｋ| 30|  未知|昭和60年|  ＳＲＣ|商業地域|  80|700|
| 中古マンション等| 千代田区|     1| 69000000|２ＬＤＫ| 70|  未知|昭和59年|  ＳＲＣ|商業地域|  80|600|
| 中古マンション等| 千代田区|     3| 21000000|  １Ｋ| 25|  未知|昭和60年|  ＳＲＣ|商業地域|  80|700|
| 中古マンション等| 千代田区|     4| 45000000|１ＬＤＫ| 45|  未知|昭和57年|  ＳＲＣ|商業地域|  80|700|
| 中古マンション等| 千代田区|     1| 66000000|２ＬＤＫ| 55|  未知|昭和59年|  ＳＲＣ|商業地域|  80|600|
| 中古マンション等| 千代田区|     5| 19000000|  １Ｋ| 20|  未知|平成15年|   ＲＣ|商業地域|  80|500|
| 中古マンション等| 千代田区|     3| 37000000|１ＬＤＫ| 45|  未知|平成24年|   ＲＣ|商業地域|  80|500|
| 中古マンション等| 千代田区|     5| 18000000|  １Ｒ| 20|  未知|平成15年|   ＲＣ|商業地域|  80|500|
|宅地(土地と建物)| 千代田区|     3|240000000|  未知| 80| 330|昭和61年|   ＲＣ|商業地域|  80|500|
| 中古マンション等| 千代田区|     1| 

In [143]:
sdf = sdf.filter(col('建築年').isNotNull())
sdf = sdf.filter(~(col('建築年').contains('戦前')))

In [144]:
sdf.select('建築年').collect()

[Row(建築年='昭和61年'),
 Row(建築年='昭和60年'),
 Row(建築年='昭和59年'),
 Row(建築年='昭和60年'),
 Row(建築年='昭和57年'),
 Row(建築年='昭和59年'),
 Row(建築年='平成15年'),
 Row(建築年='平成24年'),
 Row(建築年='平成15年'),
 Row(建築年='昭和61年'),
 Row(建築年='昭和59年'),
 Row(建築年='平成11年'),
 Row(建築年='昭和59年'),
 Row(建築年='昭和59年'),
 Row(建築年='昭和60年'),
 Row(建築年='平成19年'),
 Row(建築年='平成10年'),
 Row(建築年='平成19年'),
 Row(建築年='昭和59年'),
 Row(建築年='昭和59年'),
 Row(建築年='昭和59年'),
 Row(建築年='昭和57年'),
 Row(建築年='昭和45年'),
 Row(建築年='平成19年'),
 Row(建築年='昭和57年'),
 Row(建築年='平成19年'),
 Row(建築年='平成9年'),
 Row(建築年='平成15年'),
 Row(建築年='平成16年'),
 Row(建築年='平成16年'),
 Row(建築年='平成16年'),
 Row(建築年='平成15年'),
 Row(建築年='平成5年'),
 Row(建築年='平成24年'),
 Row(建築年='平成24年'),
 Row(建築年='昭和47年'),
 Row(建築年='平成13年'),
 Row(建築年='昭和52年'),
 Row(建築年='昭和51年'),
 Row(建築年='昭和47年'),
 Row(建築年='昭和59年'),
 Row(建築年='平成13年'),
 Row(建築年='昭和47年'),
 Row(建築年='平成25年'),
 Row(建築年='平成16年'),
 Row(建築年='昭和47年'),
 Row(建築年='平成13年'),
 Row(建築年='昭和47年'),
 Row(建築年='平成25年'),
 Row(建築年='昭和52年'),
 Row(建築年='昭和51年'),
 Row(建築年='昭和45年'),
 Row(建築年='平成13

In [145]:
import re
def year_transform(year):
    if re.match('昭和元年', year):
        year = 2018-1926
        return year
    elif re.match('昭和',year):
        year = 2018-(1925 + int(re.sub(r'\D','',year)))
        return year
    elif re.match('平成元年', year):
        year = 2018-1989
        return year
    elif re.match('平成',year):
        year = 2018-(1988 + int(re.sub(r'\D','',year)))
        return year
    else:
        return year

In [146]:
year_series = np.array([])
for i in sdf.select('建築年').collect():
#     print(year_transform(i[0]))
    np.append(year_series,year_transform(i[0]))
# year_series

In [147]:
udf_year_transform = udf(year_transform, IntegerType())
sdf = sdf.withColumn("年数", udf_year_transform("建築年"))
sdf = sdf.drop('建築年')
sdf.show(5)

+---------+-----+------+---------+----+---+----+-----+----+----+---+---+
|       種類|市区町村名|最寄駅：距離|    取引価格9| 間取り| 面積|延床面積|建物の構造|都市計画|建ぺい率|容積率| 年数|
+---------+-----+------+---------+----+---+----+-----+----+----+---+---+
|宅地(土地と建物)| 千代田区|     3|250000000|  未知| 80| 330|   ＲＣ|商業地域|  80|500| 32|
| 中古マンション等| 千代田区|     3| 22000000|  １Ｋ| 30|  未知|  ＳＲＣ|商業地域|  80|700| 33|
| 中古マンション等| 千代田区|     1| 69000000|２ＬＤＫ| 70|  未知|  ＳＲＣ|商業地域|  80|600| 34|
| 中古マンション等| 千代田区|     3| 21000000|  １Ｋ| 25|  未知|  ＳＲＣ|商業地域|  80|700| 33|
| 中古マンション等| 千代田区|     4| 45000000|１ＬＤＫ| 45|  未知|  ＳＲＣ|商業地域|  80|700| 36|
+---------+-----+------+---------+----+---+----+-----+----+----+---+---+
only showing top 5 rows



In [148]:
sdf = sdf.filter(col('容積率').isNotNull())
sdf = sdf.withColumn("延床面積", col('面積')*col('容積率')/100)

In [149]:
sdf.select('延床面積','面積','容積率').show()

+-----+---+---+
| 延床面積| 面積|容積率|
+-----+---+---+
|400.0| 80|500|
|210.0| 30|700|
|420.0| 70|600|
|175.0| 25|700|
|315.0| 45|700|
|330.0| 55|600|
|100.0| 20|500|
|225.0| 45|500|
|100.0| 20|500|
|400.0| 80|500|
|330.0| 55|600|
|275.0| 55|500|
|330.0| 55|600|
|330.0| 55|600|
|140.0| 20|700|
|350.0| 70|500|
|250.0| 50|500|
|425.0| 85|500|
|330.0| 55|600|
|330.0| 55|600|
+-----+---+---+
only showing top 20 rows



### PS  
一回目の予測ではすべての駅名などを保留し、ダミー化しました。  
そこはあんまり意味ないので、  
今回は、駅名ではなく、この駅の平均地価、この地区の平均地価などで予測してみたいです。  

In [150]:
sdf.printSchema()

root
 |-- 種類: string (nullable = true)
 |-- 市区町村名: string (nullable = true)
 |-- 最寄駅：距離: string (nullable = true)
 |-- 取引価格9: long (nullable = true)
 |-- 間取り: string (nullable = false)
 |-- 面積: string (nullable = true)
 |-- 延床面積: double (nullable = true)
 |-- 建物の構造: string (nullable = false)
 |-- 都市計画: string (nullable = true)
 |-- 建ぺい率: integer (nullable = true)
 |-- 容積率: integer (nullable = true)
 |-- 年数: integer (nullable = true)



In [151]:
sdf = sdf.withColumn("地区平米単価", col('取引価格9')/col('面積'))

city  = sdf.groupby('市区町村名').agg({"地区平米単価": "mean"})
sdf = sdf.join(city,sdf['市区町村名'] == city['市区町村名'] )


これで、市町村名が不要になる

In [152]:
sdf = sdf.drop('市区町村名')

特徴量削減のため、変な間取を削除

In [154]:
sdf.groupby('間取り').count().show(100)

+-------+-----+
|    間取り|count|
+-------+-----+
| １ＬＤＫ＋Ｓ|  130|
|   ５ＬＤＫ|   21|
|   １ＬＤＫ| 3308|
|     １Ｒ|  751|
| ３ＬＤＫ＋Ｓ|   33|
|   スタジオ|   11|
|    ５ＤＫ|    1|
|   ３ＬＤＫ|13325|
|   ２ＬＤＫ| 7848|
|   ４ＬＤＫ| 1510|
|  ２ＬＤ＋Ｓ|    1|
|     未知|33785|
|    ３ＤＫ|  569|
|    ３ＬＫ|    5|
|     ３Ｋ|   17|
|    ４ＤＫ|   11|
| ３ＬＤＫ＋Ｋ|    1|
|    ２ＬＤ|    1|
|     １Ｋ|12245|
| ４ＬＤＫ＋Ｓ|    4|
|     ４Ｋ|    2|
|  ２ＤＫ＋Ｓ|   21|
|  １ＤＫ＋Ｓ|    4|
|   １Ｌ＋Ｓ|    1|
|     ２Ｋ|  106|
|  メゾネット|    2|
|  ３ＤＫ＋Ｓ|    3|
|   ２Ｋ＋Ｓ|    1|
|    ２ＤＫ| 1679|
|オープンフロア|  334|
| ６ＬＤＫ＋Ｓ|    1|
| ２ＬＤＫ＋Ｓ|  311|
|    １ＤＫ| 1916|
|  ２ＬＫ＋Ｓ|    1|
+-------+-----+



In [155]:
floor_plan_list = ['５ＬＤＫ','スタジオ','５ＤＫ','','６ＬＤＫ＋Ｓ','メゾネット','オープンフロア','６ＬＤＫ＋Ｓ']

sdf = sdf.filter(~(col('間取り').isin(floor_plan_list)))

In [157]:
sdf.groupby('間取り').agg({"面積":"mean"}).show()

+------+------------------+
|   間取り|           avg(面積)|
+------+------------------+
|１ＬＤＫ＋Ｓ| 54.26923076923077|
|  １ＬＤＫ| 43.20737605804111|
|    １Ｒ|18.914780292942744|
|３ＬＤＫ＋Ｓ| 83.03030303030303|
|  ３ＬＤＫ| 69.84652908067542|
|  ２ＬＤＫ| 59.78656982670744|
|  ４ＬＤＫ| 88.94039735099338|
| ２ＬＤ＋Ｓ|              70.0|
|    未知|121.38582211040402|
|   ３ＤＫ| 51.48506151142355|
|   ３ＬＫ|              59.0|
|    ３Ｋ|  43.8235294117647|
|   ４ＤＫ| 65.45454545454545|
|３ＬＤＫ＋Ｋ|             155.0|
|   ２ＬＤ|              85.0|
|    １Ｋ|20.292772560228666|
|４ＬＤＫ＋Ｓ|             100.0|
|    ４Ｋ|              55.0|
| ２ＤＫ＋Ｓ| 51.19047619047619|
| １ＤＫ＋Ｓ|              42.5|
+------+------------------+
only showing top 20 rows



In [110]:
sdf.groupby('間取り').agg({"延床面積":"mean"}).show()

+------+------------------+
|   間取り|         avg(延床面積)|
+------+------------------+
|１ＬＤＫ＋Ｓ|           20525.0|
|  ５ＬＤＫ|27845.238095238095|
|  １ＬＤＫ| 17410.91293833132|
|    １Ｒ| 7680.559254327563|
|３ＬＤＫ＋Ｓ|21448.484848484848|
|  スタジオ|14045.454545454546|
|   ５ＤＫ|           33000.0|
|  ３ＬＤＫ| 18499.27204502814|
|  ２ＬＤＫ|19727.548419979612|
|  ４ＬＤＫ| 19969.27152317881|
| ２ＬＤ＋Ｓ|           21000.0|
|    未知|21240.597898475655|
|   ３ＤＫ|13324.604569420035|
|   ３ＬＫ|           12800.0|
|    ３Ｋ|           10500.0|
|   ４ＤＫ|13977.272727272728|
|３ＬＤＫ＋Ｋ|           31000.0|
|   ２ＬＤ|            8500.0|
|    １Ｋ| 8234.234381380154|
|４ＬＤＫ＋Ｓ|           20000.0|
+------+------------------+
only showing top 20 rows



pipelineなどを作って、一回予測してみましょう

その前に、もう一回スキーマを見ましょう

In [100]:
sdf.printSchema()

root
 |-- 種類: string (nullable = true)
 |-- 最寄駅：距離: string (nullable = true)
 |-- 取引価格9: long (nullable = true)
 |-- 間取り: string (nullable = false)
 |-- 面積: string (nullable = true)
 |-- 延床面積: double (nullable = true)
 |-- 建物の構造: string (nullable = false)
 |-- 都市計画: string (nullable = true)
 |-- 建ぺい率: integer (nullable = true)
 |-- 容積率: integer (nullable = true)
 |-- 年数: integer (nullable = true)
 |-- 地区平米単価: double (nullable = true)
 |-- avg(地区平米単価): double (nullable = true)



型変換

In [101]:
sdf = sdf.withColumn("最寄駅：距離", sdf["最寄駅：距離"].cast(IntegerType()))
sdf = sdf.withColumn("面積", sdf["面積"].cast(IntegerType()))

In [102]:
sdf.printSchema()


root
 |-- 種類: string (nullable = true)
 |-- 最寄駅：距離: integer (nullable = true)
 |-- 取引価格9: long (nullable = true)
 |-- 間取り: string (nullable = false)
 |-- 面積: integer (nullable = true)
 |-- 延床面積: double (nullable = true)
 |-- 建物の構造: string (nullable = false)
 |-- 都市計画: string (nullable = true)
 |-- 建ぺい率: integer (nullable = true)
 |-- 容積率: integer (nullable = true)
 |-- 年数: integer (nullable = true)
 |-- 地区平米単価: double (nullable = true)
 |-- avg(地区平米単価): double (nullable = true)



## ML

In [103]:
cols = sdf.columns
cols

['種類',
 '最寄駅：距離',
 '取引価格9',
 '間取り',
 '面積',
 '延床面積',
 '建物の構造',
 '都市計画',
 '建ぺい率',
 '容積率',
 '年数',
 '地区平米単価',
 'avg(地区平米単価)']

In [32]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
categoricalColumns = ['種類', '間取り', '建物の構造', '都市計画']
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

numericCols = ['最寄駅：距離', '面積', '延床面積', '建ぺい率', '容積率', '年数']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [33]:
sdf.show(5)

+---------+-----+-----+------+------+---------+----+---+-------+-----+----+----+---+---+
|       種類|都道府県名|市区町村名|最寄駅：名称|最寄駅：距離|    取引価格9| 間取り| 面積|   延床面積|建物の構造|都市計画|建ぺい率|容積率| 年数|
+---------+-----+-----+------+------+---------+----+---+-------+-----+----+----+---+---+
|宅地(土地と建物)|  東京都| 千代田区|   飯田橋|     3|250000000|  未知| 80|40000.0|   ＲＣ|商業地域|  80|500| 32|
| 中古マンション等|  東京都| 千代田区|   飯田橋|     3| 22000000|  １Ｋ| 30|21000.0|  ＳＲＣ|商業地域|  80|700| 33|
| 中古マンション等|  東京都| 千代田区|   飯田橋|     1| 69000000|２ＬＤＫ| 70|42000.0|  ＳＲＣ|商業地域|  80|600| 34|
| 中古マンション等|  東京都| 千代田区|   飯田橋|     3| 21000000|  １Ｋ| 25|17500.0|  ＳＲＣ|商業地域|  80|700| 33|
| 中古マンション等|  東京都| 千代田区|   飯田橋|     4| 45000000|１ＬＤＫ| 45|31500.0|  ＳＲＣ|商業地域|  80|700| 36|
+---------+-----+-----+------+------+---------+----+---+-------+-----+----+----+---+---+
only showing top 5 rows



In [34]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(sdf)
sdf = pipelineModel.transform(sdf)

In [35]:
selectedCols = ['features'] + cols
selectedCols

['features',
 '種類',
 '都道府県名',
 '市区町村名',
 '最寄駅：名称',
 '最寄駅：距離',
 '取引価格9',
 '間取り',
 '面積',
 '延床面積',
 '建物の構造',
 '都市計画',
 '建ぺい率',
 '容積率',
 '年数']

In [36]:
sdf = sdf.select(selectedCols)
sdf.printSchema()

root
 |-- features: vector (nullable = true)
 |-- 種類: string (nullable = true)
 |-- 都道府県名: string (nullable = true)
 |-- 市区町村名: string (nullable = true)
 |-- 最寄駅：名称: string (nullable = true)
 |-- 最寄駅：距離: integer (nullable = true)
 |-- 取引価格9: long (nullable = true)
 |-- 間取り: string (nullable = false)
 |-- 面積: integer (nullable = true)
 |-- 延床面積: double (nullable = true)
 |-- 建物の構造: string (nullable = false)
 |-- 都市計画: string (nullable = true)
 |-- 建ぺい率: integer (nullable = true)
 |-- 容積率: integer (nullable = true)
 |-- 年数: integer (nullable = true)



In [37]:
train,test = sdf.randomSplit([0.7,0.3],seed = 0)
print('train:',train.count())

train: 54674


モデリング

In [38]:
from pyspark.ml.regression import RandomForestRegressor

dt = RandomForestRegressor(featuresCol = 'features',labelCol = '取引価格9')
dtModel = dt.fit(train)
dt_pred = dtModel.transform(test)


In [51]:
dt_pred.select("prediction","取引価格9").show()


+--------------------+--------+
|          prediction|   取引価格9|
+--------------------+--------+
|2.5317042735197585E7|19000000|
| 3.535634116173114E7|33000000|
|3.2904317761455752E7|43000000|
|4.2324383344918735E7|69000000|
| 3.671709783440156E7|28000000|
|3.2837414280101947E7|25000000|
|  6.25192453701724E7|31000000|
|3.3906148830650195E7|21000000|
| 5.962538660011089E7|65000000|
| 4.542111223869635E7|34000000|
| 3.599818037376322E7|39000000|
|2.1556938267318584E7|23000000|
|2.1556938267318584E7|15000000|
|2.1556938267318584E7|17000000|
|2.1556938267318584E7|21000000|
|2.1556938267318584E7|20000000|
| 2.190055681944332E7|23000000|
| 2.190055681944332E7|21000000|
|2.1556938267318584E7|20000000|
|2.1556938267318584E7|21000000|
+--------------------+--------+
only showing top 20 rows



In [52]:
evaluator = RegressionEvaluator(
    labelCol="取引価格9", predictionCol="prediction", metricName="rmse")

In [53]:
evaluator.evaluate(dt_pred)

60759342.29271417

結果として、あり得ない予測です。  失敗です。。。。。  
来週は改善しましょう